## LSTM (IMDB sentiment classification)

In [1]:
import torch
import torch.nn.functional as F
from torchtext import data
from torchtext import datasets
import time
import random

In [2]:
import pandas as pd
import numpy as np

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
RANDOM_SEED = 7
torch.manual_seed(RANDOM_SEED)


## Hyper-parameters
VOCABULARY_SIZE = 20000
LEARNING_RATE = 1e-4
BATCH_SIZE = 64
NUM_EPOCHS = 10

## Model Architecture
DEVICE = DEVICE = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
EMBEDDING_DIM = 128
HIDDEN_DIM = 256
OUTPUT_DIM = 1
BIDIRECTIONAL = True
NUM_LAYERS = 2
DROPOUT = 0.5

## Data Preprocessing

In [6]:
df = pd.read_csv("data/yelp_review_polarity_csv/train.csv", header=None, index_col=None)
df.columns = ['classlabel', 'content']
df['classlabel'] = df['classlabel']-1
df.head()

In [ ]:
np.unique(df["classlabel"])

In [ ]:
np.bincount(df['classlabel'])

In [ ]:
df.loc[0:5000,['classlabel', 'content']].to_csv('data/yelp_review_polarity_csv/train_prepocessed.csv', index=None)

In [ ]:
df = pd.read_csv('data/yelp_review_polarity_csv/test.csv', header=None, index_col=None)
df.columns = ['classlabel', 'content']
df['classlabel'] = df['classlabel']-1
df.head()

In [ ]:
np.unique(df['classlabel'].values)

In [ ]:
np.bincount(df['classlabel'])

In [ ]:
df.loc[:1000,['classlabel', 'content']].to_csv('data/yelp_review_polarity_csv/test_prepocessed.csv', index=None)

In [ ]:
del df

In [ ]:
TEXT = data.Field(sequential=True,
                  tokenize='spacy',
                  include_lengths=True) # necessary for packed_padded_sequence

LABEL = data.LabelField(dtype=torch.float)

In [ ]:
fields = [('classlabel', LABEL), ('content', TEXT)]

train_dataset = data.TabularDataset(
    path = "data/yelp_review_polarity_csv/train_prepocessed.csv",
    format = "CSV",
    fields = fields,
    skip_header = True)

test_dataset = data.TabularDataset(
    path="data/yelp_review_polarity_csv/test_prepocessed.csv", 
    format='csv',
    fields = fields,
    skip_header=True)

In [ ]:
train_data, valid_data = train_dataset.split(
    split_ratio=[0.8, 0.2],
    random_state=random.seed(RANDOM_SEED))

In [ ]:
print(f'Num Train: {len(train_data)}')
print(f'Num Valid: {len(valid_data)}')

In [ ]:
TEXT.build_vocab(train_data, 
                 max_size = VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

In [ ]:
list(LABEL.vocab.freqs)[-10:]

In [ ]:
train_loader, valid_loader, test_loader = data.BucketIterator.splits(
    (train_data, valid_data, test_dataset), 
    batch_size=BATCH_SIZE,
    sort_within_batch=True, # necessary for packed_padded_sequence
    sort_key=lambda x: len(x.content),
    device=DEVICE)

In [ ]:
print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.content[0].size()}')
    print(f'Target vector size: {batch.classlabel.size()}')
    break
    
print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.content[0].size()}')
    print(f'Target vector size: {batch.classlabel.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.content[0].size()}')
    print(f'Target vector size: {batch.classlabel.size()}')
    break

## Model

In [ ]:
class RNNZrc(torch.nn.Module):
    def __init__(self, input_dim, 
                 embedding_dim, 
                 bidirectional, 
                 hidden_dim, 
                 num_layers, 
                 output_dim, 
                 dropout, 
                 pad_idx):
        
        super(RNNZrc, self).__init__()
        self.embedding = torch.nn.Embedding(input_dim, embedding_dim, padding_idx=pad_idx)
        self.rnn = torch.nn.LSTM(embedding_dim, 
                           hidden_dim,
                           num_layers=num_layers,
                           bidirectional=bidirectional, 
                           dropout=dropout)
        self.fc1 = torch.nn.Linear(hidden_dim * num_layers, 64)
        self.fc2 = torch.nn.Linear(64, output_dim)
        self.dropout = torch.nn.Dropout(dropout)
    
    def forward(self, text, text_length):

        embedded = self.dropout(self.embedding(text))
        packed_embedded = torch.nn.utils.rnn.pack_padded_sequence(embedded, text_length)
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        output, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(packed_output)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        hidden = self.fc1(hidden)
        return hidden

In [ ]:
INPUT_DIM = len(TEXT.vocab)

PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

torch.manual_seed(RANDOM_SEED)
model = RNNZrc(INPUT_DIM, EMBEDDING_DIM, BIDIRECTIONAL, HIDDEN_DIM, NUM_LAYERS, OUTPUT_DIM, DROPOUT, PAD_IDX)
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
def compute_accuracy(model, data_loader, device):
    model.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(data_loader):
            text, text_lengths = batch_data.content
            logits = model(text, text_lengths).squeeze(1)
            _, predicted_labels = torch.max(logits, 1)
            num_examples += batch_data.classlabel.size(0)
            correct_pred += (predicted_labels.long() == batch_data.classlabel.long()).sum()
        return correct_pred.float()/num_examples * 100

In [ ]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text, text_lengths = batch_data.content
        
        ### FORWARD AND BACK PROP
        logits = model(text, text_lengths).squeeze(1)
        cost = F.cross_entropy(logits, batch_data.classlabel.long())
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 1000:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Cost: {cost:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')